In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [3]:
house_df=pd.read_csv('house_price_train.csv ')
#test_df=pd.read_csv('house_price_test.csv ')

In [4]:

#house_df=pd.concat([train_df,test_df],axis=0)
house_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:



y=house_df['SalePrice']

house_df.drop(columns='SalePrice',inplace=True)




In [6]:
#house_df=house_df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [7]:
#missing value cols

missing_df=pd.DataFrame({'count':house_df.isnull().sum(),
                        'percentage':house_df.isnull().sum()/len(house_df)*100})
missing_df.sort_values(by='percentage',ascending=False,inplace=True)
missing_cols=list(missing_df[missing_df['percentage']>40].index)
missing_cols

['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']

In [8]:
#uni value cols
uni_cols=[]
for col in house_df.columns:
    if len(house_df[col].value_counts())==1 or len(house_df[col].value_counts())==len(house_df):
        uni_cols.append(col)
uni_cols

['Id']

In [9]:
house_df.drop(columns=list(set(missing_cols+uni_cols)),inplace=True)

In [10]:
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV

In [11]:
x_train,x_test,y_train,y_test=train_test_split(house_df,y,test_size=.20,random_state=42)

In [12]:
num_cols=list(house_df.select_dtypes(include=[np.number]).columns)
cat_cols=list(house_df.select_dtypes(include=[np.object]).columns)

In [13]:
for col in num_cols:
    x_train[col]=x_train[col].fillna(x_train[col].mean())
    x_test[col]=x_test[col].fillna(x_train[col].mean())

In [14]:
for col in cat_cols:
    x_train[col]=x_train[col].fillna(x_train[col].mode()[0])
    x_test[col]=x_test[col].fillna(x_train[col].mode()[0])
  

In [15]:
from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()
for col in num_cols:
    x_train[col]=scale.fit_transform(np.array(x_train[col]).reshape(-1,1))
    x_test[col]=scale.transform(np.array(x_test[col]).reshape(-1,1))

In [16]:
x_train_oe=pd.get_dummies(x_train[cat_cols])
x_test_oe=pd.get_dummies(x_test[cat_cols])

In [17]:
final_x_train_oe,final_x_test_oe=x_train_oe.align(x_test_oe,axis=1,join='inner',fill_value=0)

In [18]:
final_x_train=pd.concat([final_x_train_oe,x_train[num_cols]],axis=1)
final_x_test=pd.concat([final_x_test_oe,x_test[num_cols]],axis=1)

                         

In [19]:
print('train shape:',final_x_train_oe.shape)
print('test  shape:',final_x_test_oe.shape)

train shape: (1168, 196)
test  shape: (292, 196)


In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import classification_report,confusion_matrix,roc_curve,roc_auc_score,accuracy_score,f1_score,r2_score

C:\Users\anish\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [21]:
adaboost=AdaBoostRegressor()
adaboost.fit(final_x_train,y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [22]:
adatboost_pred=adaboost.predict(final_x_test)
print(r2_score(y_test,adatboost_pred))

0.8307916172937457


In [23]:
print('the train score is:', adaboost.score(final_x_train,y_train))
print('the test score is:', adaboost.score(final_x_test,y_test))

the train score is: 0.8655048160920503
the test score is: 0.8307916172937457
